In [1]:
import torch
print(torch.cuda.device_count())  # 打印GPU的数量
print(torch.cuda.get_device_name(0))  # 打印第一个GPU的名字
print(torch.cuda.get_device_name(1))  # 打印第二个GPU的名字（如果有）

2
Tesla T4
Tesla T4


In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="TypedStorage is deprecated")

In [3]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', k.strip()))
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False, clean_up_tokenization_spaces=True, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 如果有多个GPU
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)

model = model.to('cuda')  # 将模型分配到GPU

def summarize_text(text):
    # 预处理文本
    input_ids = tokenizer(
        [WHITESPACE_HANDLER(text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=1024
    )["input_ids"].to('cuda')  # 将输入数据移到GPU
    
    # 调用生成方法时，使用 model.module.generate
    output_ids = model.module.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [4]:
import pandas as pd
data = pd.read_csv('/kaggle/input/txtdata/runingdata121_125.csv')

from tqdm import tqdm
for i in tqdm(range(4000), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('/kaggle/working/Processeddata121_125.csv', index=False)

Processing rows: 100%|██████████| 4000/4000 [57:37<00:00,  1.16it/s]


In [5]:
import pandas as pd
data = pd.read_csv('/kaggle/input/txtdata/runingdata125_129.csv')

from tqdm import tqdm
for i in tqdm(range(4000), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('/kaggle/working/Processeddata125_129.csv', index=False)

Processing rows: 100%|██████████| 4000/4000 [55:36<00:00,  1.20it/s]


In [6]:
import pandas as pd
data = pd.read_csv('/kaggle/input/txtdata/runingdata129_133.csv')

from tqdm import tqdm
for i in tqdm(range(4000), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('/kaggle/working/Processeddata129_133.csv', index=False)

Processing rows: 100%|██████████| 4000/4000 [1:02:01<00:00,  1.07it/s]


In [7]:
import pandas as pd
data = pd.read_csv('/kaggle/input/txtdata/runingdata133_137.csv')

from tqdm import tqdm
for i in tqdm(range(4000), desc="Processing rows"):
    if data.loc[i,'text_length']>100:
        text = data.loc[i,'content']
        summary = summarize_text(text)
        data.loc[i,'summarization'] = summary
    else:
        data.loc[i,'summarization'] = data.loc[i,'content']
data.to_csv('/kaggle/working/Processeddata133_137.csv', index=False)

Processing rows: 100%|██████████| 4000/4000 [1:18:03<00:00,  1.17s/it]
